In [2]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession


In [3]:
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"stock_price_predictor"

print("bucket", default_bucket)

bucket sagemaker-us-east-1-367569554735


In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.t3.medium")

model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value="/home/sagemaker-user/user-default-efs/stock_price_predictor/google.csv",
)

mse_threshold = ParameterFloat(name="MseThreshold", default_value=0.00109616)

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "1.2-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.t3.medium",
    instance_count=processing_instance_count,
    base_job_name="sklearn-stock_price-process",
    role=role,
    sagemaker_session=pipeline_session,
)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination="/home/sagemaker-user/user-default-efs/stock_price_predictor/google.csv"),
    ],
    outputs=[
        ProcessingOutput(output_name="preprocessed", source="/home/sagemaker-user/user-default-efs/stock_price_predictor/google_preprocessed.csv")
    ],
    code="/home/sagemaker-user/user-default-efs/stock_price_predictor/preprocessing.py",
)

step_process = ProcessingStep(name="StockPricePredictor", step_args=processor_args)